In [1]:
import json
import os
import yaml

import datasets

from glob import glob

def get_yaml(fname):
    with open(fname, encoding="utf-8") as f:
        readme_lines = f.readlines()
    if '---\n' in readme_lines[1:]:
        return ''.join(readme_lines[1:readme_lines[1:].index('---\n') + 1]).strip()
    else:
        return ''

def get_dataset_confs(dataset_id):
    module_path = datasets.load.prepare_module(dataset_id, dataset=True)
    builder_cls = datasets.load.import_main_class(module_path[0], dataset=True)
    build_confs = builder_cls.BUILDER_CONFIGS
    confs = [conf.name for conf in build_confs] if len(build_confs) > 0 else ['default']
    return confs

def split_configs(tag_dict, name):
    confs = []
    for v in tag_dict.values():
        if isinstance(v, dict):
            confs = list(v.keys())
    if len(confs) == 0:
        return dict([
            (conf, tag_dict)
            for conf in get_dataset_confs(name)
        ])
    else:
        return dict([
            (k_conf, dict([
                (k, v[k_conf] if isinstance(v, dict) else v)
                for k, v in tag_dict.items()
            ]))
            for k_conf in confs
        ])

readme_files = sorted(glob("/home/yjernite/Code/datasets/datasets/*/README.md"))
len(readme_files), readme_files[:3]

(429,
 ['/home/yjernite/Code/datasets/datasets/acronym_identification/README.md',
  '/home/yjernite/Code/datasets/datasets/ade_corpus_v2/README.md',
  '/home/yjernite/Code/datasets/datasets/afrikaans_ner_corpus/README.md'])

In [2]:
readme_tags = []
missed = []

for fname in readme_files:
    if get_yaml(fname)!= '':
        try:
            readme_tags += [
                (
                    fname.split('/')[-2],
                    split_configs(yaml.load(get_yaml(fname)),fname.split('/')[-2])
                )
            ]
        except:
            print(fname)
            missed += [fname]

<ipython-input-2-f1dca04dce9f>:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  split_configs(yaml.load(get_yaml(fname)),fname.split('/')[-2])
<ipython-input-2-f1dca04dce9f>:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  split_configs(yaml.load(get_yaml(fname)),fname.split('/')[-2])


/home/yjernite/Code/datasets/datasets/multi_woz/README.md
/home/yjernite/Code/datasets/datasets/samsum/README.md


In [3]:
for name, conf_dict in readme_tags:
    saved_path = os.path.join('saved_tags', name)
    if not os.path.isdir(saved_path):
        os.mkdir(saved_path)
    for conf, tag_dict in conf_dict.items():
        conf_path = os.path.join(saved_path, conf)
        if not os.path.isdir(conf_path):
            os.mkdir(conf_path)
        json.dump(tag_dict, open(os.path.join(conf_path, 'tags.json'), 'w'))

In [41]:
readme_tags[0]

('acronym_identification',
 {'all_conf_tags': {'annotations_creators': ['expert-generated'],
   'language_creators': ['found'],
   'languages': ['en'],
   'licenses': ['mit'],
   'multilinguality': ['monolingual'],
   'size_categories': ['10K<n<100K'],
   'source_datasets': ['original'],
   'task_categories': ['structure-prediction'],
   'task_ids': ['structure-prediction-other-acronym-identification']}})